<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myXH_Risk_Reward_Ratio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import yfinance as yf
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df
cols = ['Symbol', 'ATH%', 'PE', 'EPS', 'PB', 'MCap', 'ROCE%', 'ROE%',
       'Sales_Grwth%', 'Profit_Grwth%', 'MedPE',
       'Debt2EqR', 'PAT_12M', 'CFO_2_EBITDA%', 'CapType']
myscreen_df = get_myscreen_df()[cols]

# function to fetch screener data
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df
mypps_df = get_mypps_df()
my_analysis_df = pd.merge(myscreen_df, mypps_df, on='Symbol', how='left')
my_analysis_df['Conviction']  = my_analysis_df['Conviction'] + '-' + my_analysis_df['CapType']

cols = ['Symbol', 'MCap', 'CumlRnk', 'Conviction', 'InFolio', 'ATH%']
my_analysis_df[my_analysis_df['Conviction'].isin(['X-LC','H-LC','X-MC'])][cols].sort_values(by = 'ATH%', ascending=False)

,Symbol,MCap,CumlRnk,Conviction,InFolio,ATH%
247,AWL,35526.74,58.0,X-MC,1.0,221.45
179,COLPAL,59140.55,65.0,X-MC,1.0,79.05
61,TMPV,146520.55,2.0,X-LC,1.0,78.94
127,BAJAJHFL,89672.65,63.0,X-MC,1.0,75.12
46,DLF,189671.50,40.0,H-LC,NaN,59.96
...,...,...,...,...,...,...
43,SBILIFE,199009.34,52.0,H-LC,NaN,1.74
199,MCX,48977.04,59.0,X-MC,NaN,1.73
10,LT,550296.24,22.0,X-LC,NaN,1.59
72,TORNTPHARM,129769.56,46.0,H-LC,NaN,1.30


In [2]:
import yfinance as yf

def risk_reward_52w(stock_symbol):
    try:
        stock = yf.Ticker(stock_symbol)
        hist = stock.history(period="1y")

        if hist.empty:
            return None

        current_price = hist["Close"][-1]
        high_52w = hist["High"].max()
        low_52w = hist["Low"].min()

        risk_pct = ((current_price - low_52w) / low_52w) * 100
        reward_pct = ((high_52w - current_price) / current_price) * 100
        risk_reward_ratio = reward_pct / risk_pct if risk_pct != 0 else None

        return {
            "Symbol": stock_symbol,
            "CurrentPrice": round(current_price, 2),
            "52W_High": round(high_52w, 2),
            "52W_Low": round(low_52w, 2),
            "Risk_(%)": round(risk_pct, 2),
            "Reward_(%)": round(reward_pct, 2),
            "Risk_Reward_Ratio": round(risk_reward_ratio, 2) if risk_reward_ratio else None
        }

    except Exception as e:
        return None

import pandas as pd

def risk_reward_df(stock_list):
    results = []

    for stock in stock_list:
        data = risk_reward_52w(stock + '.NS')
        if data:
            results.append(data)

    return pd.DataFrame(results)


In [3]:

stocks = my_analysis_df[my_analysis_df['Conviction'].isin(['X-LC','H-LC','X-MC'])]['Symbol'].values
df = risk_reward_df(stocks)
df['Symbol'] = df['Symbol'].str.replace('.NS', '', regex=False)
cols = ['Symbol', 'Conviction', 'InFolio', 'ATH%']
df = pd.merge(df, my_analysis_df[cols], on='Symbol', how='left')
df.sort_values(by = 'Risk_Reward_Ratio', ascending=False).head(20)

,Symbol,CurrentPrice,52W_High,52W_Low,Risk_(%),Reward_(%),Risk_Reward_Ratio,Conviction,InFolio,ATH%
60,DIXON,11238.00,18471.00,11180.00,0.52,64.36,124.06,X-MC,1.0,25.09
11,ITC,334.70,455.78,333.70,0.30,36.17,120.71,X-LC,1.0,22.11
56,INDHOTEL,678.15,855.48,672.60,0.83,26.15,31.69,X-MC,NaN,24.72
80,AWL,219.78,291.20,217.50,1.05,32.50,31.00,X-MC,1.0,221.45
73,PAGEIND,34645.00,50257.30,34055.00,1.73,45.06,26.01,X-MC,1.0,37.52
63,BAJAJHFL,94.23,136.96,92.10,2.31,45.35,19.61,X-MC,1.0,75.12
72,UBL,1533.80,2287.92,1495.40,2.57,49.17,19.15,X-MC,NaN,26.93
45,ENRIN,2404.40,3625.00,2321.20,3.58,50.77,14.16,H-LC,NaN,9.90
68,COLPAL,2106.70,2911.55,2033.00,3.63,38.20,10.54,X-MC,1.0,79.05
78,PGHH,12383.00,14759.08,12045.62,2.80,19.19,6.85,X-MC,1.0,49.39
